In [16]:
from AGNCatalogToolbox import main as agn
from colossus.cosmology import cosmology
import numpy as np

In [2]:
# my cosmology
from colossus.cosmology import cosmology
params = {'flat': True, 'H0': 70., 'Om0': 0.3, 'Ob0': 0.049, 'sigma8':0.81, 'ns':0.95}
cosmology.addCosmology('Carraro+20', params)

In [3]:
cosmo = 'Carraro+20'
cosmology = cosmology.setCosmology(cosmo)
volume = 200**3

In [4]:
z = 1

In [5]:
halos = agn.generate_semi_analytic_halo_catalogue(volume, (12, 16, 0.1), z, 0.7)
stellar_mass = agn.halo_mass_to_stellar_mass(halos, z)
black_hole_mass = agn.stellar_mass_to_black_hole_mass(stellar_mass, method="Shankar16", scatter="Intrinsic")

Scatter is a thing, valued at 0.11


In [6]:
duty_cycle = agn.to_duty_cycle("Schechter", stellar_mass, black_hole_mass, z)

Warning - we have requested redshift 1 - Selecting file Geo17DCz0.75.csv as it is closest


In [32]:
# Schechter parameters fitting
#redshift = [0.1, 1, 2]
#alpha = [-0.25,1.6,7.14]
#lambd = [0.05, -0.8,-0.5]
#alpha_pars=np.polyfit(redshift,alpha,2)
#lambda_pars=np.polyfit(redshift,lambd,2)
#np.savez('schechter_pars.npz',alpha_pars=alpha_pars,lambda_pars=lambda_pars)

schechter_pars=np.load('schechter_pars.npz')
alpha_pars=schechter_pars['alpha_pars']
lambda_pars=schechter_pars['lambda_pars']

alpha_pol=np.poly1d(alpha_pars)
lambda_pol=np.poly1d(lambda_pars)

alpha_z=alpha_pol(z)
lambda_z=lambda_pol(z)

In [7]:
luminosity = agn.black_hole_mass_to_luminosity(black_hole_mass, duty_cycle, stellar_mass, z,"Schechter",parameter1=lambda_z,
                                  parameter2=-0.65)

In [8]:
nh = agn.luminosity_to_nh(luminosity, z)
agn_type = agn.nh_to_type(nh)

In [12]:
logSFR = agn.SFR(z,stellar_mass)

In [13]:
print(logSFR[:10])

[1.4910989  0.99289613 1.38118183 0.9566826  1.4349269  1.09363132
 0.89439104 1.63980771 1.3210789  1.17825574]
